СОЗДАНИЕ ВЕКТОРНОЙ БАЗЫ ДАННЫХ CHROMADB

In [2]:
import chromadb
import os
from docx import Document

chroma_client = chromadb.Client()
collection = chroma_client.create_collection(name = "mycollection")


UniqueConstraintError: Collection mycollection already exists

ИЗВЛЕЧЕНИЕ ТЕКСТОВЫХ ДАННЫХ ИЗ ДОКУММЕНТОВ C РАСШИРЕНИЕМ .DOCX 

In [5]:

documents = []
folder_path = "documents"
for filename in os.listdir(folder_path):
    if filename.endswith('.docx'):
        file_path = os.path.join(folder_path,filename)
        doc = Document(file_path)
        text = "\n".join([paragraph.text for paragraph in doc.paragraphs ])
        documents.append(text)
ids = []
for i in range(len(documents)):
    ids.append(f'doc{i}')     
 
collection.add(
     documents=documents,
     ids = ids
)

Insert of existing embedding ID: doc0
Add of existing embedding ID: doc0


ЗАПРОС ДЛЯ RAG

In [8]:
query = "Нейросетевые реализиции методов SLAM"
results = collection.query(
    query_texts=[query],
    n_results=1
)

print("Найденные документы:",results["documents"])
print("Расстояния:",results["distances"])



Найденные документы: [['Секция 5: Методы теории автоматического управления; автоматизация и управление технологическими процессами и производствами\n\nУДК 004.93’1\nНейросетвая реализация задач SLAM\nЩеглов Владислав Андреевич\t\tshcheglovvladislav@mail.ru\nКорлякова Мария Олеговна\t\t\tmkorlyakova@yandex.ru\nРассмотрено применение нейронных сетей для решения задач одновременной локализации и картографирования. Проведен анализ существующих подходов к решению SLAM. Представлен обзор нейросетевых архитектур, успешно применяемых для решения различных аспектов SLAM, таких как детектирование особых точек и извлечение признаков изображения. Показаны преимущества нейросетевых методов в сравнении с традиционными алгоритмами.\nКлючевые слова: SLAM, нейронные сети, визуальная одометрия, построение карт\nОдновременная локализация и картографирование (SLAM) представляет собой фундаментальный метод, применяемый в области робототехники и компьютерной навигации. Этот подход позволяет роботам и автоно

ПРИВЕДЕНИЕ ТЕКСТОВЫХ ДАННЫХ К JSON ДЛЯ RETRIEVER 

In [4]:
import json

# Получение всех документов из ChromaDB
all_documents = collection.get(ids=ids)["documents"]

# Сохранение в JSON-файл
passages = [{"id": f"doc{i+1}", "text": doc} for i, doc in enumerate(all_documents)]
with open("passages.json", "w", encoding="utf-8") as f:
    json.dump(passages, f, ensure_ascii=False, indent=4)

In [5]:
from transformers import AutoTokenizer, RagRetriever, RagSequenceForGeneration
import torch

# Загрузка модели RAG
tokenizer = AutoTokenizer.from_pretrained("facebook/rag-token-base")
retriever = RagRetriever.from_pretrained("facebook/rag-token-base", index_name="exact", passages_path="passages.json")
model = RagSequenceForGeneration.from_pretrained("facebook/rag-token-base", retriever=retriever)

# # Запрос к RAG
# query = "Нейросетевые реализации методов SLAM"
# input_dict = tokenizer(query, max_length=512, truncation=True, return_tensors="pt")

# # Генерация ответа
# output = model.generate(**input_dict, retriever=retriever)
# decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)

# print(f"Ответ RAG: {decoded_output}")


c:\Users\shche\Desktop\chroma_db\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizerFast'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in une

KeyboardInterrupt: 